# Data Cleaning and Processing Project

*This project will clean and process data from a dataset which will then be better equipped to answer questions for analysis.*
    
   Tasks:
    
   - **Clean or process the salary column**
    
   - **Extract whatever information you can out of job descriptions**
    
   - **Clean and process the company name column**
    
   - **Derive new columns from existing columns(e.g. state column from the location column)**
    




## The data

In [1]:
# Importing Libraries

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
import re

In [2]:
# Original Dataset

df = pd.read_csv(r"C:\Users\gutie\Downloads\Uncleaned_DS_jobs.csv")

df.head(5)

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,0,Sr Data Scientist,$137K-$171K (Glassdoor est.),Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst\n3.1,"New York, NY","New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna"
1,1,Data Scientist,$137K-$171K (Glassdoor est.),"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech\n4.2,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1
2,2,Data Scientist,$137K-$171K (Glassdoor est.),Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group\n3.8,"Boston, MA","Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),-1
3,3,Data Scientist,$137K-$171K (Glassdoor est.),JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON\n3.5,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,2000,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),"MKS Instruments, Pfeiffer Vacuum, Agilent Tech..."
4,4,Data Scientist,$137K-$171K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"


In [3]:
# Delete columns

df = df.drop(columns=['index','Competitors','Rating','Size','Type of ownership','Industry','Sector','Revenue','Headquarters'])

In [4]:
# Delete rows with bad data

df = df[df.Founded != -1]
df = df[df.Location != 'Remote']
df = df[df.Location != 'United States']

In [5]:
# Copy main dataframe

df_Final = df.copy()

## Task 1: Clean or process the salary column

In [6]:
# Erase non-integers (Glassdoor est., $, K,) from 'Salary Estimate' column and insert new values into new column ['Salary']

df_Final["Salary"] = df_Final["Salary Estimate"].astype(str).str[:-17]
df_Final["Salary"] = df_Final["Salary"].str.replace('K', '')
df_Final["Salary"] = df_Final["Salary"].str.replace('$', '')

df_Final["Salary"]

0      137-171
1      137-171
2      137-171
3      137-171
4      137-171
        ...   
663    105-167
665    105-167
666    105-167
667    105-167
671    105-167
Name: Salary, Length: 539, dtype: object

In [7]:
# Remove Salary Estimate column

df_Final = df_Final.drop(columns=['Salary Estimate'])

### Make a Minimum, Maximum, and Average Salary Column

In [8]:
# Minimum  and Maximum Salary Column

df_Final['min_salary'] = df_Final["Salary"].str.split("-",1, expand=True)[0]
df_Final['max_salary'] = df_Final["Salary"].str.split("-",1, expand=True)[1]

df_Final[['Salary', 'min_salary', 'max_salary']].head(5)

,Salary,min_salary,max_salary
0,137-171,137,171
1,137-171,137,171
2,137-171,137,171
3,137-171,137,171
4,137-171,137,171


In [9]:
# Change Minimum and Maximum Salary column data types from object to int

df_Final['min_salary'] = df_Final['min_salary'].astype('int64')
df_Final['max_salary'] = df_Final['max_salary'].astype('int64')

In [10]:
# Make the 'Average Salary' Column

df_Final['avg_salary'] = df_Final['min_salary'] + df_Final['max_salary']
df_Final['avg_salary'] = df_Final['avg_salary'] // 2

In [11]:
# Drop 'Salary' column and move the rest of salary columns

df_Final = df_Final.drop(columns=['Salary'])

df_Final.head(5)

,Job Title,Job Description,Company Name,Location,Founded,min_salary,max_salary,avg_salary
0,Sr Data Scientist,Description\n\nThe Senior Data Scientist is re...,Healthfirst\n3.1,"New York, NY",1993,137,171,154
1,Data Scientist,"Secure our Nation, Ignite your Future\n\nJoin ...",ManTech\n4.2,"Chantilly, VA",1968,137,171,154
2,Data Scientist,Overview\n\n\nAnalysis Group is one of the lar...,Analysis Group\n3.8,"Boston, MA",1981,137,171,154
3,Data Scientist,JOB DESCRIPTION:\n\nDo you have a passion for ...,INFICON\n3.5,"Newton, MA",2000,137,171,154
4,Data Scientist,Data Scientist\nAffinity Solutions / Marketing...,Affinity Solutions\n2.9,"New York, NY",1998,137,171,154


## Task 2: Extract whatever information you can out of job descriptions

### Extract information from the Job Description about the required or preferred technologies/tools for the Job 

In [12]:
# Create multiple columns to denote whether the particular technology/tool is mentioned in the Job Description

# python
df_Final = df_Final.assign(python=0)
# excel
df_Final = df_Final.assign(excel=0)
# sql
df_Final = df_Final.assign(sql=0)
# aws
df_Final = df_Final.assign(aws=0)
# big_data
df_Final = df_Final.assign(big_data=0)
# tableau
df_Final = df_Final.assign(tableau=0)

In [13]:
# If the technology/tool is mentioned in the Job Description, change cell value to 'Required'

# Python
conditions = [
    (df['Job Description'].str.contains('python')),
    (df['Job Description'].str.contains('Python')),
    (df['Job Description'].str.contains('PYTHON'))
    ]

values = ['Required', 'Required', 'Required']

df_Final['python'] = np.select(conditions, values)


# Excel
conditions = [
    (df['Job Description'].str.contains('excel')),
    (df['Job Description'].str.contains('Excel')),
    (df['Job Description'].str.contains('EXCEL'))
    ]

values = ['Required', 'Required', 'Required']

df_Final['excel'] = np.select(conditions, values)


# SQL
conditions = [
    (df['Job Description'].str.contains('SQL')),
    (df['Job Description'].str.contains('sql')),
    (df['Job Description'].str.contains('Sql'))
    ]

values = ['Required', 'Required', 'Required']

df_Final['sql'] = np.select(conditions, values)


# AWS
conditions = [
    (df['Job Description'].str.contains('AWS')),
    (df['Job Description'].str.contains('Aws')),
    (df['Job Description'].str.contains('aws'))
    ]

values = ['Required', 'Required', 'Required']

df_Final['aws'] = np.select(conditions, values)


# Big Data
conditions = [
    (df['Job Description'].str.contains('Big-data')),
    (df['Job Description'].str.contains('Big-Data')),
    (df['Job Description'].str.contains('BIG-DATA')),
    (df['Job Description'].str.contains('Big Data')),
    (df['Job Description'].str.contains('Big data')),
    (df['Job Description'].str.contains('BIG DATA')),
    (df['Job Description'].str.contains('big data')),
    (df['Job Description'].str.contains('big-data'))
    ]

values = ['Required', 'Required', 'Required', 'Required', 'Required', 'Required', 'Required', 'Required']

df_Final['big_data'] = np.select(conditions, values)


# Tableau
conditions = [
    (df['Job Description'].str.contains('Tableau')),
    (df['Job Description'].str.contains('tableau')),
    (df['Job Description'].str.contains('TABLEAU')),
    ]

values = ['Required', 'Required', 'Required']

df_Final['tableau'] = np.select(conditions, values)

df_Final[['python', 'excel', 'sql', 'aws', 'big_data', 'tableau']].head(10)

,python,excel,sql,aws,big_data,tableau
0,0,0,0,Required,0,0
1,0,0,Required,0,Required,0
2,Required,Required,0,Required,0,0
3,Required,Required,Required,Required,0,0
4,Required,Required,Required,0,0,0
5,Required,Required,Required,0,Required,0
6,Required,0,0,0,0,0
7,Required,0,Required,0,0,0
8,0,0,0,0,0,0
9,Required,0,Required,0,0,0


In [14]:
# Drop 'Job Description' column

df_Final = df_Final.drop(columns=['Job Description'])

df_Final

,Job Title,Company Name,Location,Founded,min_salary,max_salary,avg_salary,python,excel,sql,aws,big_data,tableau
0,Sr Data Scientist,Healthfirst\n3.1,"New York, NY",1993,137,171,154,0,0,0,Required,0,0
1,Data Scientist,ManTech\n4.2,"Chantilly, VA",1968,137,171,154,0,0,Required,0,Required,0
2,Data Scientist,Analysis Group\n3.8,"Boston, MA",1981,137,171,154,Required,Required,0,Required,0,0
3,Data Scientist,INFICON\n3.5,"Newton, MA",2000,137,171,154,Required,Required,Required,Required,0,0
4,Data Scientist,Affinity Solutions\n2.9,"New York, NY",1998,137,171,154,Required,Required,Required,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
663,Data Scientist,A-Line Staffing Solutions\n4.1,"Durham, NC",2004,105,167,136,Required,0,Required,0,Required,0
665,Data Scientist,"Criterion Systems, Inc.\n3.8","Vienna, VA",2005,105,167,136,Required,Required,Required,0,0,0
666,Data Scientist,Foundation Medicine\n4.0,"Boston, MA",2010,105,167,136,Required,0,Required,0,0,0
667,Data Scientist,TRANZACT\n3.6,"Fort Lee, NJ",1989,105,167,136,Required,Required,Required,0,Required,Required


## Task 3: Clean and process the company name column

In [15]:
# Remove numbers, \n, and . from the 'Company Name' column

df_Final['Company Name'] = df_Final['Company Name'].str.replace('\d+', '')
df_Final['Company Name'] = df_Final['Company Name'].str.replace('\n', '')
df_Final['Company Name'] = df_Final['Company Name'].str.replace('.', '')

df_Final['Company Name']

0                    Healthfirst
1                        ManTech
2                 Analysis Group
3                        INFICON
4             Affinity Solutions
                 ...            
663    A-Line Staffing Solutions
665       Criterion Systems, Inc
666          Foundation Medicine
667                     TRANZACT
671                    --Flowers
Name: Company Name, Length: 539, dtype: object

### Replace Founded Column with Company_Age Column

In [16]:
# Make 'Company Age' Column and Remove 'Founded' column

df_Final['Company_Age'] = 2023 - df_Final['Founded']
df_Final = df_Final.drop(columns=['Founded'])

In [17]:
# Move 'Company Age' column

df_Final.insert(3,'Company_Age', df_Final.pop('Company_Age'))

df_Final

,Job Title,Company Name,Location,Company_Age,min_salary,max_salary,avg_salary,python,excel,sql,aws,big_data,tableau
0,Sr Data Scientist,Healthfirst,"New York, NY",30,137,171,154,0,0,0,Required,0,0
1,Data Scientist,ManTech,"Chantilly, VA",55,137,171,154,0,0,Required,0,Required,0
2,Data Scientist,Analysis Group,"Boston, MA",42,137,171,154,Required,Required,0,Required,0,0
3,Data Scientist,INFICON,"Newton, MA",23,137,171,154,Required,Required,Required,Required,0,0
4,Data Scientist,Affinity Solutions,"New York, NY",25,137,171,154,Required,Required,Required,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
663,Data Scientist,A-Line Staffing Solutions,"Durham, NC",19,105,167,136,Required,0,Required,0,Required,0
665,Data Scientist,"Criterion Systems, Inc","Vienna, VA",18,105,167,136,Required,Required,Required,0,0,0
666,Data Scientist,Foundation Medicine,"Boston, MA",13,105,167,136,Required,0,Required,0,0,0
667,Data Scientist,TRANZACT,"Fort Lee, NJ",34,105,167,136,Required,Required,Required,0,Required,Required


## Task 4: Derive new columns from existing columns(e.g. state column from the location column)

### Creating State column derived from Location column

In [18]:
# Create 'State' and City columns
# You can stop here if you're satisfied with the states' abbreviations

df_Final["State"] = df_Final['Location'].str[-2:]
df_Final["City"] = df_Final['Location'].str[:-4]

df_Final["State"].head(5)

0    NY
1    VA
2    MA
3    MA
4    NY
Name: State, dtype: object

In [19]:
# Make a dictionary giving the full state name for each state abbreviation

abbrevs = {
    'AK': 'Alaska', 'AL': 'Alabama', 'AR': 'Arkansas', 'AZ': 'Arizona', 'CA': 'California', 'CO': 'Colorado',
    'CT': 'Connecticut', 'DC': 'District of Columbia', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia',
    'HI': 'Hawaii', 'IA': 'Iowa', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'KS': 'Kansas',
    'KY': 'Kentucky', 'LA': 'Louisiana', 'MA': 'Massachusetts', 'MD': 'Maryland', 'ME': 'Maine', 'MI': 'Michigan',
    'MN': 'Minnesota', 'MO': 'Missouri', 'MS': 'Mississippi', 'MT': 'Montana', 'NC': 'North Carolina', 'ND': 'North Dakota',
    'NE': 'Nebraska', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NV': 'Nevada', 'NY': 'New York',
    'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
    'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VA': 'Virginia', 'VT': 'Vermont',
    'WA': 'Washington', 'WI': 'Wisconsin', 'WV': 'West Virginia', 'WY': 'Wyoming', 'es': 'United States', 'te': 'Remote',
    'as': 'Texas'
}

In [20]:
# Now replace the abbreviations with the full state name and drop the 'Locations' column

df_Final["State"].replace(abbrevs, inplace=True)
df_Final = df_Final.drop(columns=['Location'])

In [21]:
# Move 'State' and 'City' columns

df_Final.insert(2,'State', df_Final.pop('State'))
df_Final.insert(3,'City', df_Final.pop('City'))

df_Final.head(5)

,Job Title,Company Name,State,City,Company_Age,min_salary,max_salary,avg_salary,python,excel,sql,aws,big_data,tableau
0,Sr Data Scientist,Healthfirst,New York,New York,30,137,171,154,0,0,0,Required,0,0
1,Data Scientist,ManTech,Virginia,Chantilly,55,137,171,154,0,0,Required,0,Required,0
2,Data Scientist,Analysis Group,Massachusetts,Boston,42,137,171,154,Required,Required,0,Required,0,0
3,Data Scientist,INFICON,Massachusetts,Newton,23,137,171,154,Required,Required,Required,Required,0,0
4,Data Scientist,Affinity Solutions,New York,New York,25,137,171,154,Required,Required,Required,0,0,0


### Creating Seniority column based on the Job Title column

In [22]:
# Create the seniority column

df_Final = df_Final.assign(Seniority=None)

In [23]:
# Condition to replace value with either Senior or Junior

conditions = [
    (df['Job Title'].str.contains('Sr')),
    (df['Job Title'].str.contains('Senior')),
    (df['Job Title'].str.contains('Principal')),
    (df['Job Title'].str.contains('Lead')),
    (df['Job Title'].str.contains('Sr.')),
    (df['Job Title'].str.contains('VP')),
    (df['Job Title'].str.contains('Vice President')),
    (df['Job Title'].str.contains('Manager')),
    (df['Job Title'].str.contains('Jr')),
    (df['Job Title'].str.contains('Junior')),
    (df['Job Title'].str.contains('Associate')),
    (df['Job Title'].str.contains('Jr.'))
    ]

values = ['Senior', 'Senior', 'Senior', 'Senior', 'Senior', 'Senior', 'Senior', 'Senior', 
          'Junior', 'Junior', 'Junior', 'Junior']


df_Final['Seniority'] = np.select(conditions, values)

In [24]:
# Move seniority row and show all rows with either Senior or Junior in 'seniority' column

df_Final.insert(2,'Seniority', df_Final.pop('Seniority'))

df_Final[df_Final['Seniority'].str.contains("nior")]

,Job Title,Company Name,Seniority,State,City,Company_Age,min_salary,max_salary,avg_salary,python,excel,sql,aws,big_data,tableau
0,Sr Data Scientist,Healthfirst,Senior,New York,New York,30,137,171,154,0,0,0,Required,0,0
33,Senior Research Statistician- Data Scientist,Acuity Insurance,Senior,Wisconsin,Sheboygan,98,75,131,103,0,0,0,0,0,0
35,Associate Data Scientist,Puget Sound Energy,Junior,Washington,Bothell,24,75,131,103,0,Required,Required,0,0,0
39,Senior Analyst/Data Scientist,Edmundscom,Senior,California,Santa Monica,57,75,131,103,Required,Required,Required,Required,0,Required
46,Senior Data Scientist,Klaviyo,Senior,Massachusetts,Boston,11,75,131,103,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,Senior Data Scientist,Autodesk,Senior,California,San Francisco,41,80,132,106,Required,0,Required,0,Required,0
611,(Sr.) Data Scientist -,Merrick Bank,Senior,ah,,26,87,141,114,Required,Required,Required,Required,0,0
612,Senior Data Scientist,ManTech,Senior,Virginia,Alexandria,55,87,141,114,Required,0,Required,0,Required,Required
652,Senior Data Scientist,GutCheck,Senior,Colorado,Denver,14,105,167,136,0,0,0,0,0,0


### Format

In [25]:
# Format salary columns to have $ and K

df_Final.head(10).style.format({"Company_Age":"{0}yrs","min_salary":"${0}K","max_salary":"${0:}K","avg_salary":"${0:}K"})

,Job Title,Company Name,Seniority,State,City,Company_Age,min_salary,max_salary,avg_salary,python,excel,sql,aws,big_data,tableau
0,Sr Data Scientist,Healthfirst,Senior,New York,New York,30yrs,$137K,$171K,$154K,0,0,0,Required,0,0
1,Data Scientist,ManTech,0,Virginia,Chantilly,55yrs,$137K,$171K,$154K,0,0,Required,0,Required,0
2,Data Scientist,Analysis Group,0,Massachusetts,Boston,42yrs,$137K,$171K,$154K,Required,Required,0,Required,0,0
3,Data Scientist,INFICON,0,Massachusetts,Newton,23yrs,$137K,$171K,$154K,Required,Required,Required,Required,0,0
4,Data Scientist,Affinity Solutions,0,New York,New York,25yrs,$137K,$171K,$154K,Required,Required,Required,0,0,0
5,Data Scientist,HG Insights,0,California,Santa Barbara,13yrs,$137K,$171K,$154K,Required,Required,Required,0,Required,0
6,Data Scientist / Machine Learning Expert,Novartis,0,Massachusetts,Cambridge,27yrs,$137K,$171K,$154K,Required,0,0,0,0,0
7,Data Scientist,iRobot,0,Massachusetts,Bedford,33yrs,$137K,$171K,$154K,Required,0,Required,0,0,0
8,Staff Data Scientist - Analytics,Intuit - Data,0,California,San Diego,40yrs,$137K,$171K,$154K,0,0,0,0,0,0
9,Data Scientist,XSELL Technologies,0,Illinois,Chicago,9yrs,$137K,$171K,$154K,Required,0,Required,0,0,0
